In [43]:
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix
from mixed_naive_bayes import MixedNB

import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns

# from datetime import datetime

# matplotlib inline

In [2]:
df = pd.read_csv('recent-grads-new-features.csv')
df.head()

,Unnamed: 0,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,...,Major_category_Education,Major_category_Engineering,Major_category_Health,Major_category_Humanities & Liberal Arts,Major_category_Industrial Arts & Consumer Services,Major_category_Interdisciplinary,Major_category_Law & Public Policy,Major_category_Physical Sciences,Major_category_Psychology & Social Work,Major_category_Social Science
0,0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,...,0,1,0,0,0,0,0,0,0,0
1,1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,...,0,1,0,0,0,0,0,0,0,0
2,2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,...,0,1,0,0,0,0,0,0,0,0
3,3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,...,0,1,0,0,0,0,0,0,0,0
4,4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,...,0,1,0,0,0,0,0,0,0,0


In [3]:
features = df[["Major_category", "Num_majors", "Women", "ShareWomen", "Percent_college", "Percent_non_college", "Percent_low_wage", "Percent_sample_size", "Unemployment_rate", "P75th", "P25th", "Part_time"]]

medians = df[["Median_category"]]

In [37]:
medians

,Median_category
0,110-120K
1,70-80K
2,70-80K
3,70-80K
4,60-70K
...,...
167,20-30K
168,20-30K
169,20-30K
170,20-30K


In [5]:
le = LabelEncoder()
le.fit(df["Major_category"])
# le.transform(features["Major_category"])

LabelEncoder()

In [6]:
features["Major_category"] = le.transform(features["Major_category"])

/srv/jupyter_python3-extras/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
features.head()

,Major_category,Num_majors,Women,ShareWomen,Percent_college,Percent_non_college,Percent_low_wage,Percent_sample_size,Unemployment_rate,P75th,P25th,Part_time
0,7,29.0,282,0.120564,0.655836,0.155622,0.082514,0.015391,0.018381,125000,95000,270
1,7,29.0,77,0.101852,0.462963,0.339947,0.066138,0.009259,0.117241,90000,55000,170
2,7,29.0,131,0.153037,0.532710,0.205607,0.000000,0.003505,0.024096,105000,50000,133
3,7,29.0,135,0.107313,0.420509,0.081081,0.000000,0.012719,0.050125,80000,43000,150
4,7,29.0,11021,0.341631,0.567700,0.137632,0.030130,0.008958,0.061098,75000,50000,5180


In [35]:
### Categorical 60%

X_train, X_test, y_train, y_test = train_test_split(features, medians, test_size=0.2, random_state=42)
naive_bayes = CategoricalNB()
naive_bayes.fit(X_train, y_train)
predictions = naive_bayes.predict(X_test)
naive_bayes.score(X_test, y_test)

/srv/jupyter_python3-extras/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.6

In [39]:
### Gaussian 68%

X_train, X_test, y_train, y_test = train_test_split(features, medians, test_size=0.2, random_state=42)
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
predictions = naive_bayes.predict(X_test)
naive_bayes.score(X_test, y_test)

/srv/jupyter_python3-extras/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.6857142857142857

In [46]:
## Mixed Naive Bayes --> 60% accuracy
## https://pypi.org/project/mixed-naive-bayes/#api-documentation
## https://github.com/remykarem/mixed-naive-bayes

label_encoder = LabelEncoder()
mixedNB_y_train = label_encoder.fit_transform(y_train["Median_category"])
mixedNB_y_test = label_encoder.transform(y_test["Median_category"])

mixedNB = MixedNB(categorical_features=[0])
mixedNB.fit(X_train, mixedNB_y_train)
mixedNB_predictions = mixedNB.predict(X_test)

mixedNB.score(X_test, mixedNB_y_test)
print(confusion_matrix(mixedNB_y_test, mixedNB_predictions), ": is the confusion matrix")

[[ 2  0  0  0  0]
 [ 3 12  4  0  0]
 [ 0  0  1  1  2]
 [ 0  0  0  5  3]
 [ 0  0  0  1  1]] : is the confusion matrix


In [33]:
mixedNB_predictions_inversed = label_encoder.inverse_transform(mixedNB_predictions)

In [34]:
mixedNB_predictions_inversed

array(['40-50K', '30-40K', '20-30K', '60-70K', '20-30K', '20-30K',
       '40-50K', '20-30K', '30-40K', '50-60K', '30-40K', '40-50K',
       '40-50K', '40-50K', '30-40K', '60-70K', '30-40K', '50-60K',
       '30-40K', '30-40K', '50-60K', '60-70K', '60-70K', '30-40K',
       '20-30K', '60-70K', '50-60K', '30-40K', '50-60K', '50-60K',
       '30-40K', '60-70K', '50-60K', '30-40K', '30-40K'], dtype=object)

In [44]:
### param tuning, 68.7%
### https://medium.com/analytics-vidhya/how-to-improve-naive-bayes-9fa698e14cba

param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
nbModel_grid.fit(X_train, y_train)
# print(nbModel_grid.best_estimator_)
y_pred = nbModel_grid.predict(X_test)
nbModel_grid.score(X_test, y_test)
print(confusion_matrix(y_test, y_pred), ": is the confusion matrix")


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/srv/jupyter_python3-extras/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 516 tasks      | elapsed:    0.8s


[[ 2  0  0  0  0]
 [ 3 14  2  0  0]
 [ 0  0  1  1  2]
 [ 0  0  0  6  2]
 [ 0  0  0  1  1]] : is the confusion matrix


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    1.5s finished
/srv/jupyter_python3-extras/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [25]:
mixedNB_y_test

array([1, 1, 1, 2, 1, 0, 1, 1, 1, 3, 1, 1, 2, 1, 1, 3, 1, 3, 1, 1, 3, 3,
       2, 1, 0, 3, 3, 1, 3, 4, 1, 4, 2, 1, 1])

In [26]:
predictions

array(['40-50K', '30-40K', '30-40K', '30-40K', '20-30K', '20-30K',
       '30-40K', '30-40K', '30-40K', '50-60K', '30-40K', '30-40K',
       '40-50K', '30-40K', '30-40K', '50-60K', '30-40K', '40-50K',
       '40-50K', '30-40K', '30-40K', '60-70K', '30-40K', '30-40K',
       '30-40K', '30-40K', '50-60K', '30-40K', '40-50K', '50-60K',
       '30-40K', '60-70K', '50-60K', '30-40K', '40-50K'], dtype='<U8')

In [27]:
y_test

,Median_category
76,30-40K
138,30-40K
152,30-40K
60,40-50K
156,30-40K
165,20-30K
85,30-40K
142,30-40K
114,30-40K
31,50-60K
